In [1]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user
!pip install tqdm

  Created wheel for landlord-ai: filename=landlord_ai-0.1.16-py3-none-any.whl size=14900 sha256=71389c3b96dc9e234101f890bd072a385c97d87361591701e789ad4fe6f52f54
  Stored in directory: /home/jupyter/.cache/pip/wheels/f2/30/a1/6679bde3a13e99296208bc091c3494be0e7ec695f2382e7894
Successfully built landlord-ai
  Attempting uninstall: landlord-ai
    Found existing installation: landlord-ai 0.1.14
    Uninstalling landlord-ai-0.1.14:
      Successfully uninstalled landlord-ai-0.1.14
  Using cached Keras_Preprocessing-1.1.0-py2.py3-none-any.whl (41 kB)


In [2]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *
import os

import pickle
import random
from tqdm import tqdm

from landlordai.game.player import LearningPlayer_v1

Using TensorFlow backend.


In [3]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, path_ids, batch_size=1024, shuffle=True, clamp=False, timesteps_length=LearningPlayer_v1.TIMESTEPS):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.path_ids = path_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.clamp = clamp
        self.timesteps_length = timesteps_length
        
        self.load_cache()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return 1000

    def load_cache(self):
        with open(random.choice(self.path_ids), 'rb') as f:
            history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
            
            if self.shuffle:
                p = np.random.permutation(len(history_matrices))
                
                history_matrices = np.array(history_matrices)[p]
                move_vectors = move_vectors[p]
                hand_vectors = hand_vectors[p]
                y = y[p]
        
        # unflatten
        history_matrices = self.densify(history_matrices)

        self.cache = (history_matrices, move_vectors, hand_vectors, y) 
        self.curr_index = 0
        
    
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        limit = min(len(self.cache[0]), (self.curr_index + 1) * self.batch_size)
        
        #print(self.curr_index * self.batch_size, limit)
        history_matrices = self.cache[0][self.curr_index * self.batch_size: limit]
        move_vectors = self.cache[1][self.curr_index * self.batch_size: limit]
        hand_vectors = self.cache[2][self.curr_index * self.batch_size: limit]
        #print(self.curr_index * self.batch_size, limit)
        y = self.cache[3][self.curr_index * self.batch_size: limit]
        self.curr_index += 1
        
        # load a new batch
        if (self.curr_index + 1) * self.batch_size >= len(self.cache[0]):
            self.load_cache()
        
        return [history_matrices, move_vectors, hand_vectors], self.adjust_y(y)

    def densify(self, sparse_matrix):
        return np.array([x.todense()[:self.timesteps_length] for x in sparse_matrix])

    def adjust_y(self, y):
        if not self.clamp:
            return y
        new_y = []
        for elem in y:
            if abs(int(elem) - elem) > 1E-4:
                new_y.append(0)
            else:
                new_y.append(elem)
        return np.array(new_y)



In [4]:
data_folder = '4_1_sim3'

In [5]:
assert data_folder is not None
!rm -r ../data/$data_folder
!gsutil -m cp -r gs://landlord_ai/$data_folder ../data/

Copying gs://landlord_ai/4_1_sim3/0.pkl...
Copying gs://landlord_ai/4_1_sim3/1.pkl...
Copying gs://landlord_ai/4_1_sim3/10.pkl...
Copying gs://landlord_ai/4_1_sim3/11.pkl...
Copying gs://landlord_ai/4_1_sim3/12.pkl...
Copying gs://landlord_ai/4_1_sim3/13.pkl...
Copying gs://landlord_ai/4_1_sim3/14.pkl...
Copying gs://landlord_ai/4_1_sim3/15.pkl...ne                                   
Copying gs://landlord_ai/4_1_sim3/16.pkl...ne                                   
Copying gs://landlord_ai/4_1_sim3/17.pkl...ne                                   
Copying gs://landlord_ai/4_1_sim3/18.pkl...
Copying gs://landlord_ai/4_1_sim3/2.pkl...one                                   
Copying gs://landlord_ai/4_1_sim3/19.pkl...
Copying gs://landlord_ai/4_1_sim3/21.pkl...ne                                   
Copying gs://landlord_ai/4_1_sim3/20.pkl...
Copying gs://landlord_ai/4_1_sim3/22.pkl...ne                                   
Copying gs://landlord_ai/4_1_sim3/23.pkl...ne                               

In [6]:

directory = '../data/' + data_folder
filenames = [directory + '/' + file for file in os.listdir(directory)]

if len(filenames) == 1:
    train_path_ids = filenames
    test_path_ids = filenames
else:
    divider = int(len(filenames) * 0.9)
    train_path_ids = filenames[:divider]
    test_path_ids = filenames[divider:]

In [8]:
# quality control
inspector = DataGenerator(train_path_ids, shuffle=False)

In [87]:
irregularities = 0
for j in tqdm(range(100)):
    a_set = inspector[j]
    for i in range(1000):
        history = a_set[0][0][i]
        history_sum = np.sum(history)
        q = a_set[1][i]
        irregular = abs(q) > 1 and history_sum < 20
        if irregular:
            #with open('irregular.pkl', 'wb') as f:
            #    pickle.dump((a_set, i), f)
            irregularities += 1
            print(history, q)
        #assert not (abs(q) > 1 and history_sum < 10)

100%|██████████| 100/100 [00:05<00:00, 17.96it/s]


In [7]:
train_gen = DataGenerator(train_path_ids, timesteps_length=50)
test_gen = DataGenerator(test_path_ids, timesteps_length=50)

In [8]:
assert not np.allclose(train_gen[0][0][0], train_gen[0][0][0])
assert len(train_gen[0][0]) == 3
for i in range(3):
    get_set = train_gen[0][0][0]
    if len(get_set.shape) != 3:
        print(get_set)


In [9]:
def create_model_bidi():
    K.clear_session()
    GRU_DIM = 32

    history_inp = Input((None, LearningPlayer_v1.TIMESTEP_FEATURES), name='history_inp')
    move_inp = Input((LearningPlayer_v1.TIMESTEP_FEATURES, ), name='move_inp')
    hand_inp = Input((LearningPlayer_v1.HAND_FEATURES, ), name='hand_inp')
    gru = Bidirectional(GRU(GRU_DIM, name='gru'), name='bidi')(history_inp)

    concat = Concatenate()([gru, move_inp, hand_inp])
    hidden = Dense(32, activation='relu', name='hidden')(concat)

    output = Dense(1, activation='linear', name='output')(BatchNormalization(name='bn')(hidden))
    combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
    combined_net.compile(loss=keras.losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])
    return combined_net

In [10]:
import subprocess
def has_layer(model, layer):
    try:
        model.get_layer(layer)
        return True
    except:
        return False

def split_model(composite, model_folder):
    best_model = keras.models.load_model(composite)
    
    if has_layer(best_model, 'bn2'):
        split_model_2(best_model, model_folder)
    elif has_layer(best_model, 'bidi'):
        split_model_bidi(best_model, model_folder)
    else:
        split_model_1(best_model, model_folder)
    
def split_model_bidi(best_model, model_folder):
    bn = best_model.get_layer('bn')
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('bidi').output])

    vector_history_inp = Input((best_model.get_layer('bidi').output.shape[1], ), name='vector_history_inp')
    
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden = best_model.get_layer('hidden')(concat)
    output = best_model.get_layer('output')(bn(hidden))

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))

In [11]:
def sanity_check_model(combined_file, net_dir):
    sanity_set = train_gen[0]
    historical_features, move_vectors, hand_vectors = sanity_set[0]
    targets = sanity_set[1]

    player = LearningPlayer_v1(name='sanity', net_dir=str(net_dir))
    
    historical_matrix = player.history_net.predict(historical_features, batch_size=1024)

    from sklearn import metrics
    
    error_1 = metrics.mean_squared_error(targets, player.get_position_predictions(historical_matrix, move_vectors, hand_vectors))
    
    composite = keras.models.load_model(combined_file)
    error_2 = metrics.mean_squared_error(targets, composite.predict([historical_features, move_vectors, hand_vectors], batch_size=1024))
    print(combined_file, error_1, error_2)
    assert np.abs(error_1 - error_2) < 1E-2

In [12]:
from pathlib import Path

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()
    
def publish_model(i):
    combined_file = data_folder + '_combined_' + str(i) + '.h5'
    if os.path.exists(combined_file):
        model_folder_name = data_folder + '_model' + str(i)

        model_folder_path = Path('../models/', model_folder_name)
        delete_dir(model_folder_path)
        model_folder_path.mkdir()

        split_model(combined_file, model_folder_path)
        sanity_check_model(combined_file, model_folder_path)
        print(model_folder_name)
        subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name + '/*', 'gs://landlord_ai/models/' + model_folder_name])

In [13]:
def train_model(fname='model.h5'):
    combined_net = create_model_bidi()
    
    callbacks = [
        EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=2),
        ModelCheckpoint(fname, monitor='val_mean_squared_error', mode='min', verbose=1, save_best_only=True)
    ]

    combined_net.fit_generator(train_gen,
              epochs=50,
                steps_per_epoch=2000,
                validation_steps=200,
                callbacks=callbacks,
                validation_data=test_gen,
                shuffle=True,
                workers=2,
                max_queue_size=200,
                use_multiprocessing=False
              )
    return combined_net

for i in range(8, 10):
    train_model(data_folder + '_combined_' + str(i) + '.h5')
    publish_model(i)

Epoch 1/50
   2/2000 [..............................] - ETA: 1:04:24 - loss: 4.7898 - mean_squared_error: 4.7898

/opt/conda/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.480550). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


2000/2000 [==============================] - 292s 146ms/step - loss: 0.8816 - mean_squared_error: 0.8814 - val_loss: 0.3146 - val_mean_squared_error: 0.7094

Epoch 00001: val_mean_squared_error improved from inf to 0.70940, saving model to 4_1_sim3_combined_8.h5
Epoch 2/50
2000/2000 [==============================] - 286s 143ms/step - loss: 0.5180 - mean_squared_error: 0.5180 - val_loss: 0.4422 - val_mean_squared_error: 0.6416

Epoch 00002: val_mean_squared_error improved from 0.70940 to 0.64162, saving model to 4_1_sim3_combined_8.h5
Epoch 3/50
2000/2000 [==============================] - 289s 144ms/step - loss: 0.4432 - mean_squared_error: 0.4432 - val_loss: 0.4311 - val_mean_squared_error: 0.6006

Epoch 00003: val_mean_squared_error improved from 0.64162 to 0.60061, saving model to 4_1_sim3_combined_8.h5
Epoch 4/50
2000/2000 [==============================] - 285s 142ms/step - loss: 0.3856 - mean_squared_error: 0.3857 - val_loss: 0.4870 - val_mean_squared_error: 0.4622

Epoch 00004:

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


ValueError: Error when checking input: expected vector_history_inp to have shape (64,) but got array with shape (1,)

In [14]:
publish_model(8)

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_1_sim3_combined_8.h5 0.2535367960371706 0.2535367960371706
4_1_sim3_model8
